# Macro

* Rust currently offers two macro systems: 
  * the regular macro system, **declarative macros**, available in Rust by default, 
  * and **procedural macros**, which require a flag to enable.

## Declarative Macros

* A macro definition starts with `macro_rules!` followed by the name of the macro:

In [24]:
macro_rules! noop_macro {
    () => {};
}

In [25]:
noop_macro!();

* A macro defined with `macro_rules!` works entirely by pattern matching. The body of a macro is just a series of rules:

    ```rust
    ( pattern1 ) => ( template1 );
    ( pattern2 ) => ( template2 );
    ```
* We can match the different kinds of code constructs in Rust, also known as *code fragments* or *patterns*, and then generate new code based on the matched pattern:

In [11]:
macro_rules! describe_what_it_is {
    () => {
        println!("A macro with no arguments.");
    };
    ($e:expr) => {
        println!("A macro with an expression.");
    };
    ($s:stmt) => {
        println!("A macro with a statement.");
    };
}

In [12]:
describe_what_it_is!();
describe_what_it_is!(42 * 665);
describe_what_it_is!(;);

A macro with no arguments.
A macro with an expression.
A macro with a statement.


* Fragments can be any type of code construct in Rust so long as it’s valid syntactically. You can also match multiple arguments:

In [20]:
macro_rules! describe_what_it_is {
    ($e:expr, $s:stmt) => {
        println!("An expression followed by a statement")
    };
}

In [22]:
describe_what_it_is!(42,;);

An expression followed by a statement


In [26]:
describe_what_it_is!(;,;);

Error: no rules expected the token `;`

Error: unused macro definition: `noop_macro`

### Example - `check_eq!` (like `assert_eq!`)

In [48]:
macro_rules! check_eq {
    ($left:expr, $right:expr) => ({
        match (&$left, &$right) {
            (left_val, right_val) => {
                if !(*left_val == *right_val) {
                    panic!("assertion failed: `(left == right)`\n  left: `{:?}`,\n right: `{:?}`", *left_val, *right_val);
                }
            }
        }
    });
}

In [52]:
check_eq!(2 * 8, 16);

In [53]:
check_eq!(2 * 8, 17);

thread '<unnamed>' panicked at src/lib.rs:25:1:
assertion failed: `(left == right)`
  left: `16`,
 right: `17`
stack backtrace:
   0: std::panicking::begin_panic_handler
             at /rustc/9b00956e56009bab2aa15d7bff10916599e3d6d6/library\std\src\panicking.rs:645
   1: core::panicking::panic_fmt
             at /rustc/9b00956e56009bab2aa15d7bff10916599e3d6d6/library\core\src\panicking.rs:72
   2: <unknown>
   3: <unknown>
   4: <unknown>
   5: <unknown>
   6: <unknown>
   7: <unknown>
   8: <unknown>
   9: <unknown>
  10: <unknown>
  11: <unknown>
  12: BaseThreadInitThunk
  13: RtlUserThreadStart
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


### Repetition

Repetition is a powerful feature of macros. It allows you to match a pattern zero or more times. The syntax for repetition is `$( PATTERN )*`.

|   Pattern    |                    Meaning                     |
| ------------ | ---------------------------------------------- |
| `$( ... )*`  | Match 0 or more times with no separator        |
| `$( ... ),*` | Match 0 or more times, separated by commas     |
| `$( ... );*` | Match 0 or more times, separated by semicolons |
| `$( ... )+`  | Match 1 or more times with no separator        |
| `$( ... ),+` | Match 1 or more times, separated by commas     |
| `$( ... );+` | Match 1 or more times, separated by semicolons |

In [4]:
macro_rules! create_vec {
    ($($item:expr),*) => {
        {
            let mut temp_vec = Vec::new();
            $(temp_vec.push($item);)*
            temp_vec
        }
    };
}

In [7]:
let v: Vec<i32> = create_vec![1, 2, 3, 4, 5];
v

[1, 2, 3, 4, 5]

### Built-in Macros

* `file!` - expands to string literal: the current filename
* `line!` - expands to u32 literal: the current line number (starting from 1)
* `column!` - expands to u32 literal: the current column (starting from 0)

In [17]:
println!("File: {}, Line: {}, Column: {}", file!(), line!(), column!());

File: src/lib.rs, Line: 116, Column: 62


* `stringify!` - converts any token into a string.

In [21]:
macro_rules! describe_expression {
    ($e:expr) => {
        println!("The expression is: {} = {}", stringify!($e), $e);
    };
}

In [23]:
describe_expression!(4 * 7);

The expression is: 4 * 7 = 28


* `concat!(str0, str1, ...)` expands to a single string literal made by concatenating its arguments.

In [25]:
concat!("abc", "def")

"abcdef"

* `env!("VAR_NAME") `expands to a string: the value of the specified environment variable at compile time. If the variable doesn’t exist, it’s a compilation error.

In [29]:
env!("PATH")

"C:\\Users\\kp\\AppData\\Local\\Temp\\.tmpDymWYg\\target\\x86_64-pc-windows-msvc\\debug\\deps;C:\\Users\\kp\\AppData\\Local\\Temp\\.tmpDymWYg\\target\\debug\\deps;C:\\Users\\kp\\.rustup\\toolchains\\stable-x86_64-pc-windows-msvc\\lib\\rustlib\\x86_64-pc-windows-msvc\\lib;C:\\Program Files (x86)\\Common Files\\Oracle\\Java\\javapath;C:\\Python310\\Scripts\\;C:\\Python310\\;C:\\Program Files (x86)\\Microsoft SDKs\\Azure\\CLI2\\wbin;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\;C:\\Windows\\System32\\OpenSSH\\;C:\\Program Files\\Microsoft SQL Server\\130\\Tools\\Binn\\;C:\\Program Files (x86)\\NVIDIA Corporation\\PhysX\\Common;c:\\Program Files\\LLVM\\bin\\;C:\\WINDOWS\\system32;C:\\WINDOWS;C:\\WINDOWS\\System32\\Wbem;C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0\\;C:\\WINDOWS\\System32\\OpenSSH\\;C:\\Program Files\\Intel\\WiFi\\bin\\;C:\\Program Files\\Common Files\\Intel\\WirelessCommon\\;C:\\Program Files\\nodejs\\;C:\\P

* `option_env!("VAR_NAME")` is the same as env! except that it returns an `Option<&'static str>` that is `None` if the specified variable is not set.

In [30]:
option_env!("HOME")

None

* `include!("file.rs")` expands to the contents of the specified file, which must be valid Rust code — either an expression or a sequence of items.

* `include_str!("file.txt")` expands to a `&'static str` containing the text of the specified file.

* `include_bytes!("file.txt")` expands to a `&'static [u8]` containing the binary contents of the specified file.

## Importing and Exporting Macros

Since macros are expanded early in compilation, before Rust knows the full module structure of your project, they aren’t imported and exported in the usual way.

* Within a single crate:

  * Macros that are visible in one module are automatically visible in its child modules.
  * To export macros from a module “upward” to its parent module, use the `#[macro_use]` attribute. For example, suppose our `lib.rs` looks like this:

    ```rust
    #[macro_use] mod macros;
    mod client;
    mod server;
    ```

  * All macros defined in the macros module are imported into `lib.rs` and therefore visible throughout the rest of the crate, including in client and server.

* When working with multiple crates:
  * To import macros from another crate, use `#[macro_use]` on the extern crate declaration.
  * To export macros from your crate, mark each public macro with `#[macro_export]`.